In [2]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import text_normalizer as tn
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
data = fetch_20newsgroups(subset='all', shuffle=True,
                          remove=('headers', 'footers', 'quotes'))
data_labels_map = dict(enumerate(data.target_names))

In [3]:
corpus, target_labels, target_names = (data.data, data.target, 
                                       [data_labels_map[label] for label in data.target])
data_df = pd.DataFrame({'Article': corpus, 'Target Label': target_labels, 'Target Name': target_names})
print(data_df.shape)
data_df.head(10)

(18846, 3)


,Article,Target Label,Target Name
0,\n\nI am sure some bashers of Pens fans are pr...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,4,comp.sys.mac.hardware
5,\n\nBack in high school I worked as a lab assi...,12,sci.electronics
6,\n\nAE is in Dallas...try 214/241-6060 or 214/...,4,comp.sys.mac.hardware
7,"\n[stuff deleted]\n\nOk, here's the solution t...",10,rec.sport.hockey
8,"\n\n\nYeah, it's the second one. And I believ...",10,rec.sport.hockey
9,\nIf a Christian means someone who believes in...,19,talk.religion.misc


In [4]:
total_nulls = data_df[data_df.Article.str.strip() == ''].shape[0]
print("Empty documents:", total_nulls)

Empty documents: 515


In [5]:
data_df = data_df[~(data_df.Article.str.strip() == '')]
data_df.shape

(18331, 3)

In [9]:
import nltk
stopword_list = nltk.corpus.stopwords.words('english')
# just to keep negation if any in bi-grams
stopword_list.remove('no')
stopword_list.remove('not')

# normalize our corpus
norm_corpus = tn.normalize_corpus(corpus=data_df['Article'], html_stripping=True, contraction_expansion=True, 
                                  accented_char_removal=True, text_lower_case=True, text_lemmatization=True, 
                                  text_stemming=False, special_char_removal=True, remove_digits=True,
                                  stopword_removal=True, stopwords=stopword_list)
data_df['Clean Article'] = norm_corpus

In [10]:
data_df = data_df[['Article', 'Clean Article', 'Target Label', 'Target Name']]
data_df.head(10)

,Article,Clean Article,Target Label,Target Name
0,\n\nI am sure some bashers of Pens fans are pr...,sure basher pen fan pretty confused lack kind ...,10,rec.sport.hockey
1,My brother is in the market for a high-perform...,brother market high performance video card sup...,3,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,finally say dream mediterranean new area great...,17,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think scsi card dma transfer not disk scsi car...,3,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,old jasmine drive not use new system understan...,4,comp.sys.mac.hardware
5,\n\nBack in high school I worked as a lab assi...,back high school work lab assistant bunch expe...,12,sci.electronics
6,\n\nAE is in Dallas...try 214/241-6060 or 214/...,ae dallas try tech support may line one get start,4,comp.sys.mac.hardware
7,"\n[stuff deleted]\n\nOk, here's the solution t...",stuff delete ok solution problem move canada y...,10,rec.sport.hockey
8,"\n\n\nYeah, it's the second one. And I believ...",yeah second one believe price try get good loo...,10,rec.sport.hockey
9,\nIf a Christian means someone who believes in...,christian mean someone believe divinity jesus ...,19,talk.religion.misc


In [11]:
data_df = data_df.replace(r'^(\s?)+$', np.nan, regex=True)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18331 entries, 0 to 18845
Data columns (total 4 columns):
Article          18331 non-null object
Clean Article    18304 non-null object
Target Label     18331 non-null int64
Target Name      18331 non-null object
dtypes: int64(1), object(3)
memory usage: 1.3+ MB


In [28]:
data_df = data_df.dropna().reset_index(drop=True)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18304 entries, 0 to 18303
Data columns (total 4 columns):
Article          18304 non-null object
Clean Article    18304 non-null object
Target Label     18304 non-null int64
Target Name      18304 non-null object
dtypes: int64(1), object(3)
memory usage: 572.1+ KB


In [29]:
data_df.to_csv('clean_newsgroups.csv', index=False)

In [3]:
data_df = pd.read_csv('clean_newsgroups.csv')

In [7]:
from sklearn.model_selection import train_test_split

train_corpus, test_corpus, train_label_nums, test_label_nums, train_label_names, test_label_names =\
                                 train_test_split(np.array(data_df['Clean Article']), np.array(data_df['Target Label']),
                                                       np.array(data_df['Target Name']), test_size=0.33, random_state=42)

train_corpus.shape, test_corpus.shape

((12263,), (6041,))

In [8]:
from collections import Counter

trd = dict(Counter(train_label_names))
tsd = dict(Counter(test_label_names))

(pd.DataFrame([[key, trd[key], tsd[key]] for key in trd], 
             columns=['Target Label', 'Train Count', 'Test Count'])
.sort_values(by=['Train Count', 'Test Count'],
             ascending=False))

,Target Label,Train Count,Test Count
19,comp.windows.x,693,287
0,rec.sport.hockey,666,308
1,comp.graphics,664,289
6,sci.crypt,660,302
5,soc.religion.christian,653,321
9,sci.electronics,649,307
13,misc.forsale,645,314
2,comp.sys.ibm.pc.hardware,639,324
10,sci.med,638,322
17,comp.sys.mac.hardware,632,295


In [105]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score

# build BOW features on train articles
cv = CountVectorizer(binary=False, min_df=0.0, max_df=1.0)
cv_train_features = cv.fit_transform(train_corpus)

In [106]:
# transform test articles into features
cv_test_features = cv.transform(test_corpus)

In [107]:
print('BOW model:> Train features shape:', cv_train_features.shape, ' Test features shape:', cv_test_features.shape)

BOW model:> Train features shape: (12263, 66865)  Test features shape: (6041, 66865)


In [108]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB(alpha=1)
mnb.fit(cv_train_features, train_label_names)
mnb_bow_cv_scores = cross_val_score(mnb, cv_train_features, train_label_names, cv=5)
mnb_bow_cv_mean_score = np.mean(mnb_bow_cv_scores)
print('CV Accuracy (5-fold):', mnb_bow_cv_scores)
print('Mean CV Accuracy:', mnb_bow_cv_mean_score)
mnb_bow_test_score = mnb.score(cv_test_features, test_label_names)
print('Test Accuracy:', mnb_bow_test_score)

CV Accuracy (5-fold): [ 0.68468102  0.68241042  0.67835304  0.67741935  0.6792144 ]
Mean CV Accuracy: 0.680415648396
Test Accuracy: 0.680185399768


In [109]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2', max_iter=100, C=1, random_state=42)
lr.fit(cv_train_features, train_label_names)
lr_bow_cv_scores = cross_val_score(lr, cv_train_features, train_label_names, cv=5)
lr_bow_cv_mean_score = np.mean(lr_bow_cv_scores)
print('CV Accuracy (5-fold):', lr_bow_cv_scores)
print('Mean CV Accuracy:', lr_bow_cv_mean_score)
lr_bow_test_score = lr.score(cv_test_features, test_label_names)
print('Test Accuracy:', lr_bow_test_score)

CV Accuracy (5-fold): [ 0.70418529  0.69788274  0.69384427  0.6998775   0.69599018]
Mean CV Accuracy: 0.698355996012
Test Accuracy: 0.703856977322


In [110]:
from sklearn.svm import LinearSVC

svm = LinearSVC(penalty='l2', C=1, random_state=42)
svm.fit(cv_train_features, train_label_names)
svm_bow_cv_scores = cross_val_score(svm, cv_train_features, train_label_names, cv=5)
svm_bow_cv_mean_score = np.mean(svm_bow_cv_scores)
print('CV Accuracy (5-fold):', svm_bow_cv_scores)
print('Mean CV Accuracy:', svm_bow_cv_mean_score)
svm_bow_test_score = svm.score(cv_test_features, test_label_names)
print('Test Accuracy:', svm_bow_test_score)

CV Accuracy (5-fold): [ 0.64120276  0.64169381  0.64900122  0.64107799  0.63993453]
Mean CV Accuracy: 0.642582064348
Test Accuracy: 0.656679357722


In [111]:
from sklearn.linear_model import SGDClassifier

svm_sgd = SGDClassifier(loss='hinge', penalty='l2', max_iter=5, random_state=42)
svm_sgd.fit(cv_train_features, train_label_names)
svmsgd_bow_cv_scores = cross_val_score(svm_sgd, cv_train_features, train_label_names, cv=5)
svmsgd_bow_cv_mean_score = np.mean(svmsgd_bow_cv_scores)
print('CV Accuracy (5-fold):', svmsgd_bow_cv_scores)
print('Mean CV Accuracy:', svmsgd_bow_cv_mean_score)
svmsgd_bow_test_score = svm_sgd.score(cv_test_features, test_label_names)
print('Test Accuracy:', svmsgd_bow_test_score)

CV Accuracy (5-fold): [ 0.66030069  0.62459283  0.65185487  0.63209473  0.64157119]
Mean CV Accuracy: 0.642082864709
Test Accuracy: 0.633007780169


In [121]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=10, random_state=42)
rfc.fit(cv_train_features, train_label_names)
rfc_bow_cv_scores = cross_val_score(rfc, cv_train_features, train_label_names, cv=5)
rfc_bow_cv_mean_score = np.mean(rfc_bow_cv_scores)
print('CV Accuracy (5-fold):', rfc_bow_cv_scores)
print('Mean CV Accuracy:', rfc_bow_cv_mean_score)
rfc_bow_test_score = rfc.score(cv_test_features, test_label_names)
print('Test Accuracy:', rfc_bow_test_score)

CV Accuracy (5-fold): [ 0.52052011  0.51669381  0.53485528  0.51327072  0.5212766 ]
Mean CV Accuracy: 0.521323304518
Test Accuracy: 0.52987915908


In [122]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=10, random_state=42)
gbc.fit(cv_train_features, train_label_names)
gbc_bow_cv_scores = cross_val_score(gbc, cv_train_features, train_label_names, cv=5)
gbc_bow_cv_mean_score = np.mean(gbc_bow_cv_scores)
print('CV Accuracy (5-fold):', gbc_bow_cv_scores)
print('Mean CV Accuracy:', gbc_bow_cv_mean_score)
gbc_bow_test_score = gbc.score(cv_test_features, test_label_names)
print('Test Accuracy:', gbc_bow_test_score)

CV Accuracy (5-fold): [ 0.55424624  0.53827362  0.54219323  0.55206207  0.55441899]
Mean CV Accuracy: 0.548238828239
Test Accuracy: 0.547922529383


In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer

# build BOW features on train articles
tv = TfidfVectorizer(use_idf=True, min_df=0.0, max_df=1.0)
tv_train_features = tv.fit_transform(train_corpus)

In [113]:
# transform test articles into features
tv_test_features = tv.transform(test_corpus)

In [114]:
print('TFIDF model:> Train features shape:', tv_train_features.shape, ' Test features shape:', tv_test_features.shape)

TFIDF model:> Train features shape: (12263, 66865)  Test features shape: (6041, 66865)


In [115]:
mnb = MultinomialNB(alpha=1)
mnb.fit(tv_train_features, train_label_names)
mnb_tfidf_cv_scores = cross_val_score(mnb, tv_train_features, train_label_names, cv=5)
mnb_tfidf_cv_mean_score = np.mean(mnb_tfidf_cv_scores)
print('CV Accuracy (5-fold):', mnb_tfidf_cv_scores)
print('Mean CV Accuracy:', mnb_tfidf_cv_mean_score)
mnb_tfidf_test_score = mnb.score(tv_test_features, test_label_names)
print('Test Accuracy:', mnb_tfidf_test_score)

CV Accuracy (5-fold): [ 0.71759447  0.70969055  0.71585813  0.7121274   0.7111293 ]
Mean CV Accuracy: 0.713279971122
Test Accuracy: 0.713954643271


In [116]:
lr = LogisticRegression(penalty='l2', max_iter=100, C=1, random_state=42)
lr.fit(tv_train_features, train_label_names)
lr_tfidf_cv_scores = cross_val_score(lr, tv_train_features, train_label_names, cv=5)
lr_tfidf_cv_mean_score = np.mean(lr_tfidf_cv_scores)
print('CV Accuracy (5-fold):', lr_tfidf_cv_scores)
print('Mean CV Accuracy:', lr_tfidf_cv_mean_score)
lr_tfidf_test_score = lr.score(tv_test_features, test_label_names)
print('Test Accuracy:', lr_tfidf_test_score)

CV Accuracy (5-fold): [ 0.74725721  0.73493485  0.73257236  0.74520212  0.73076923]
Mean CV Accuracy: 0.738147156079
Test Accuracy: 0.745240854163


In [117]:
svm = LinearSVC(penalty='l2', C=1, random_state=42)
svm.fit(tv_train_features, train_label_names)
svm_tfidf_cv_scores = cross_val_score(svm, tv_train_features, train_label_names, cv=5)
svm_tfidf_cv_mean_score = np.mean(svm_tfidf_cv_scores)
print('CV Accuracy (5-fold):', svm_tfidf_cv_scores)
print('Mean CV Accuracy:', svm_tfidf_cv_mean_score)
svm_tfidf_test_score = svm.score(tv_test_features, test_label_names)
print('Test Accuracy:', svm_tfidf_test_score)

CV Accuracy (5-fold): [ 0.76635514  0.7536645   0.75743987  0.76439363  0.75695581]
Mean CV Accuracy: 0.75976178901
Test Accuracy: 0.762456546929


In [118]:
svm_sgd = SGDClassifier(loss='hinge', penalty='l2', max_iter=5, random_state=42)
svm_sgd.fit(tv_train_features, train_label_names)
svmsgd_tfidf_cv_scores = cross_val_score(svm_sgd, tv_train_features, train_label_names, cv=5)
svmsgd_tfidf_cv_mean_score = np.mean(svmsgd_tfidf_cv_scores)
print('CV Accuracy (5-fold):', svmsgd_tfidf_cv_scores)
print('Mean CV Accuracy:', svmsgd_tfidf_cv_mean_score)
svmsgd_tfidf_test_score = svm_sgd.score(tv_test_features, test_label_names)
print('Test Accuracy:', svmsgd_tfidf_test_score)

CV Accuracy (5-fold): [ 0.76513612  0.75570033  0.75377089  0.76112699  0.75695581]
Mean CV Accuracy: 0.75853802856
Test Accuracy: 0.765767257077


In [123]:
rfc = RandomForestClassifier(n_estimators=10, random_state=42)
rfc.fit(tv_train_features, train_label_names)
rfc_tfidf_cv_scores = cross_val_score(rfc, tv_train_features, train_label_names, cv=5)
rfc_tfidf_cv_mean_score = np.mean(rfc_tfidf_cv_scores)
print('CV Accuracy (5-fold):', rfc_tfidf_cv_scores)
print('Mean CV Accuracy:', rfc_tfidf_cv_mean_score)
rfc_tfidf_test_score = rfc.score(tv_test_features, test_label_names)
print('Test Accuracy:', rfc_tfidf_test_score)

CV Accuracy (5-fold): [ 0.53596099  0.5252443   0.53852426  0.51204573  0.54296236]
Mean CV Accuracy: 0.53094752738
Test Accuracy: 0.545936103294


In [124]:
gbc = GradientBoostingClassifier(n_estimators=10, random_state=42)
gbc.fit(tv_train_features, train_label_names)
gbc_tfidf_cv_scores = cross_val_score(gbc, tv_train_features, train_label_names, cv=5)
gbc_tfidf_cv_mean_score = np.mean(gbc_tfidf_cv_scores)
print('CV Accuracy (5-fold):', gbc_tfidf_cv_scores)
print('Mean CV Accuracy:', gbc_tfidf_cv_mean_score)
gbc_tfidf_test_score = gbc.score(tv_test_features, test_label_names)
print('Test Accuracy:', gbc_tfidf_test_score)

CV Accuracy (5-fold): [ 0.55790329  0.53827362  0.55768447  0.55859535  0.54541735]
Mean CV Accuracy: 0.551574813725
Test Accuracy: 0.548584671412


In [125]:
pd.DataFrame([['Naive Bayes', mnb_bow_cv_mean_score, mnb_bow_test_score, 
               mnb_tfidf_cv_mean_score, mnb_tfidf_test_score],
              ['Logistic Regression', lr_bow_cv_mean_score, lr_bow_test_score, 
               lr_tfidf_cv_mean_score, lr_tfidf_test_score],
              ['Linear SVM', svm_bow_cv_mean_score, svm_bow_test_score, 
               svm_tfidf_cv_mean_score, svm_tfidf_test_score],
              ['Linear SVM (SGD)', svmsgd_bow_cv_mean_score, svmsgd_bow_test_score, 
               svmsgd_tfidf_cv_mean_score, svmsgd_tfidf_test_score],
              ['Random Forest', rfc_bow_cv_mean_score, rfc_bow_test_score, 
               rfc_tfidf_cv_mean_score, rfc_tfidf_test_score],
              ['Gradient Boosted Machines', gbc_bow_cv_mean_score, gbc_bow_test_score, 
               gbc_tfidf_cv_mean_score, gbc_tfidf_test_score]],
             columns=['Model', 'CV Score (TF)', 'Test Score (TF)', 'CV Score (TF-IDF)', 'Test Score (TF-IDF)'],
             ).T

,0,1,2,3,4,5
Model,Naive Bayes,Logistic Regression,Linear SVM,Linear SVM (SGD),Random Forest,Gradient Boosted Machines
CV Score (TF),0.680416,0.698356,0.642582,0.642083,0.521323,0.548239
Test Score (TF),0.680185,0.703857,0.656679,0.633008,0.529879,0.547923
CV Score (TF-IDF),0.71328,0.738147,0.759762,0.758538,0.530948,0.551575
Test Score (TF-IDF),0.713955,0.745241,0.762457,0.765767,0.545936,0.548585


In [11]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

mnb_pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                        ('mnb', MultinomialNB())
                       ])

param_grid = {'tfidf__ngram_range': [(1, 1), (1, 2)],
              'mnb__alpha': [1e-5, 1e-4, 1e-2, 1e-1, 1]
}

gs_mnb = GridSearchCV(mnb_pipeline, param_grid, cv=5, verbose=2)
gs_mnb = gs_mnb.fit(train_corpus, train_label_names)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] mnb__alpha=1e-05, tfidf__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ...... mnb__alpha=1e-05, tfidf__ngram_range=(1, 1), total=   1.5s
[CV] mnb__alpha=1e-05, tfidf__ngram_range=(1, 1) .....................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


[CV] ...... mnb__alpha=1e-05, tfidf__ngram_range=(1, 1), total=   1.5s
[CV] mnb__alpha=1e-05, tfidf__ngram_range=(1, 1) .....................
[CV] ...... mnb__alpha=1e-05, tfidf__ngram_range=(1, 1), total=   1.4s
[CV] mnb__alpha=1e-05, tfidf__ngram_range=(1, 1) .....................
[CV] ...... mnb__alpha=1e-05, tfidf__ngram_range=(1, 1), total=   1.4s
[CV] mnb__alpha=1e-05, tfidf__ngram_range=(1, 1) .....................
[CV] ...... mnb__alpha=1e-05, tfidf__ngram_range=(1, 1), total=   1.5s
[CV] mnb__alpha=1e-05, tfidf__ngram_range=(1, 2) .....................
[CV] ...... mnb__alpha=1e-05, tfidf__ngram_range=(1, 2), total=   6.3s
[CV] mnb__alpha=1e-05, tfidf__ngram_range=(1, 2) .....................
[CV] ...... mnb__alpha=1e-05, tfidf__ngram_range=(1, 2), total=   6.4s
[CV] mnb__alpha=1e-05, tfidf__ngram_range=(1, 2) .....................
[CV] ...... mnb__alpha=1e-05, tfidf__ngram_range=(1, 2), total=   6.6s
[CV] mnb__alpha=1e-05, tfidf__ngram_range=(1, 2) .....................
[CV] .

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  4.7min finished


In [12]:
gs_mnb.best_estimator_.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
           vocabulary=None)),
  ('mnb', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))],
 'tfidf': TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
         stop_words=None, strip_accents=None, sublinear_tf=False,
         token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=No

In [24]:
cv_results = gs_mnb.cv_results_
results_df = pd.DataFrame({'rank': cv_results['rank_test_score'],
                           'params': cv_results['params'], 
                           'cv score (mean)': cv_results['mean_test_score'], 
                           'cv score (std)': cv_results['std_test_score']} 
              )
results_df = results_df.sort_values(by=['rank'], ascending=True)
pd.set_option('display.max_colwidth', 100)
results_df

,cv score (mean),cv score (std),params,rank
5,0.773710,0.007229,"{'mnb__alpha': 0.01, 'tfidf__ngram_range': (1, 2)}",1
4,0.768654,0.003437,"{'mnb__alpha': 0.01, 'tfidf__ngram_range': (1, 1)}",2
6,0.762945,0.003709,"{'mnb__alpha': 0.1, 'tfidf__ngram_range': (1, 1)}",3
3,0.757400,0.006856,"{'mnb__alpha': 0.0001, 'tfidf__ngram_range': (1, 2)}",4
7,0.754383,0.006655,"{'mnb__alpha': 0.1, 'tfidf__ngram_range': (1, 2)}",5
1,0.749246,0.006356,"{'mnb__alpha': 1e-05, 'tfidf__ngram_range': (1, 2)}",6
2,0.742233,0.006627,"{'mnb__alpha': 0.0001, 'tfidf__ngram_range': (1, 1)}",7
0,0.730816,0.007727,"{'mnb__alpha': 1e-05, 'tfidf__ngram_range': (1, 1)}",8
8,0.714996,0.002637,"{'mnb__alpha': 1, 'tfidf__ngram_range': (1, 1)}",9
9,0.704314,0.003039,"{'mnb__alpha': 1, 'tfidf__ngram_range': (1, 2)}",10


In [13]:
best_mnb_test_score = gs_mnb.score(test_corpus, test_label_names)
print('Test Accuracy :', best_mnb_test_score)

Test Accuracy : 0.7735474259228604


In [15]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC

In [97]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

lr_pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                        ('lr', LogisticRegression(penalty='l2', max_iter=100, random_state=42))
                       ])

param_grid = {'tfidf__ngram_range': [(1, 1), (1, 2)],
              'lr__C': [1, 5, 10]
}

gs_lr = GridSearchCV(lr_pipeline, param_grid, cv=5, verbose=2)
gs_lr = gs_lr.fit(train_corpus, train_label_names)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] lr__C=1, tfidf__ngram_range=(1, 1) ..............................
[CV] ............... lr__C=1, tfidf__ngram_range=(1, 1), total=   5.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.1s remaining:    0.0s


[CV] lr__C=1, tfidf__ngram_range=(1, 1) ..............................
[CV] ............... lr__C=1, tfidf__ngram_range=(1, 1), total=   6.5s
[CV] lr__C=1, tfidf__ngram_range=(1, 1) ..............................
[CV] ............... lr__C=1, tfidf__ngram_range=(1, 1), total=   5.9s
[CV] lr__C=1, tfidf__ngram_range=(1, 1) ..............................
[CV] ............... lr__C=1, tfidf__ngram_range=(1, 1), total=   6.2s
[CV] lr__C=1, tfidf__ngram_range=(1, 1) ..............................
[CV] ............... lr__C=1, tfidf__ngram_range=(1, 1), total=   6.1s
[CV] lr__C=1, tfidf__ngram_range=(1, 2) ..............................
[CV] ............... lr__C=1, tfidf__ngram_range=(1, 2), total=  27.4s
[CV] lr__C=1, tfidf__ngram_range=(1, 2) ..............................
[CV] ............... lr__C=1, tfidf__ngram_range=(1, 2), total=  27.7s
[CV] lr__C=1, tfidf__ngram_range=(1, 2) ..............................
[CV] ............... lr__C=1, tfidf__ngram_range=(1, 2), total=  28.2s
[CV] l

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 13.0min finished


In [98]:
gs_lr.best_estimator_

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
 ...alty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [99]:
best_lr_test_score = gs_lr.score(test_corpus, test_label_names)
print('Test Accuracy :', best_lr_test_score)

Test Accuracy : 0.766926005628


In [83]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

svm_pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                        ('svm', LinearSVC(random_state=42))
                       ])

param_grid = {'tfidf__ngram_range': [(1, 1), (1, 2)],
              'svm__C': [0.01, 0.1, 1, 5]
}

gs_svm = GridSearchCV(svm_pipeline, param_grid, cv=5, verbose=2)
gs_svm = gs_svm.fit(train_corpus, train_label_names)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] tfidf__ngram_range=(1, 1), svm__C=0.01 ..........................
[CV] ........... tfidf__ngram_range=(1, 1), svm__C=0.01, total=   2.7s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.0s remaining:    0.0s


[CV] tfidf__ngram_range=(1, 1), svm__C=0.01 ..........................
[CV] ........... tfidf__ngram_range=(1, 1), svm__C=0.01, total=   3.1s
[CV] tfidf__ngram_range=(1, 1), svm__C=0.01 ..........................
[CV] ........... tfidf__ngram_range=(1, 1), svm__C=0.01, total=   3.0s
[CV] tfidf__ngram_range=(1, 1), svm__C=0.01 ..........................
[CV] ........... tfidf__ngram_range=(1, 1), svm__C=0.01, total=   2.9s
[CV] tfidf__ngram_range=(1, 1), svm__C=0.01 ..........................
[CV] ........... tfidf__ngram_range=(1, 1), svm__C=0.01, total=   3.3s
[CV] tfidf__ngram_range=(1, 2), svm__C=0.01 ..........................
[CV] ........... tfidf__ngram_range=(1, 2), svm__C=0.01, total=  11.6s
[CV] tfidf__ngram_range=(1, 2), svm__C=0.01 ..........................
[CV] ........... tfidf__ngram_range=(1, 2), svm__C=0.01, total=  12.5s
[CV] tfidf__ngram_range=(1, 2), svm__C=0.01 ..........................
[CV] ........... tfidf__ngram_range=(1, 2), svm__C=0.01, total=  11.3s
[CV] t

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  8.3min finished


In [84]:
gs_svm.best_estimator_.get_params()

{'memory': None,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
           stop_words=None, strip_accents=None, sublinear_tf=False,
           token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
           vocabulary=None)),
  ('svm', LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
        intercept_scaling=1, loss='squared_hinge', max_iter=1000,
        multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
        verbose=0))],
 'svm': LinearSVC(C=5, class_weight=None, dual=True, fit_intercept=True,
      intercept_scaling=1, loss='squared_hinge', max_iter=1000,
      multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
      verbose=0),
 'svm__C': 5,
 'svm__class_weight': No

In [85]:
best_svm_test_score = gs_svm.score(test_corpus, test_label_names)
print('Test Accuracy :', best_svm_test_score)

Test Accuracy : 0.77685813607


In [67]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

sgd_pipeline = Pipeline([('tfidf', TfidfVectorizer()),
                        ('sgd', SGDClassifier(random_state=42))
                       ])

param_grid = {'tfidf__ngram_range': [(1, 1), (1, 2)],
              'sgd__alpha': [1e-7, 1e-6, 1e-5, 1e-4]
}

gs_sgd = GridSearchCV(sgd_pipeline, param_grid, cv=5, verbose=2)
gs_sgd = gs_sgd.fit(train_corpus, train_label_names)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] tfidf__ngram_range=(1, 1), sgd__alpha=1e-07 .....................
[CV] ...... tfidf__ngram_range=(1, 1), sgd__alpha=1e-07, total=   2.0s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.2s remaining:    0.0s


[CV] tfidf__ngram_range=(1, 1), sgd__alpha=1e-07 .....................
[CV] ...... tfidf__ngram_range=(1, 1), sgd__alpha=1e-07, total=   2.3s
[CV] tfidf__ngram_range=(1, 1), sgd__alpha=1e-07 .....................
[CV] ...... tfidf__ngram_range=(1, 1), sgd__alpha=1e-07, total=   3.4s
[CV] tfidf__ngram_range=(1, 1), sgd__alpha=1e-07 .....................
[CV] ...... tfidf__ngram_range=(1, 1), sgd__alpha=1e-07, total=   3.0s
[CV] tfidf__ngram_range=(1, 1), sgd__alpha=1e-07 .....................
[CV] ...... tfidf__ngram_range=(1, 1), sgd__alpha=1e-07, total=   2.3s
[CV] tfidf__ngram_range=(1, 2), sgd__alpha=1e-07 .....................
[CV] ...... tfidf__ngram_range=(1, 2), sgd__alpha=1e-07, total=   9.0s
[CV] tfidf__ngram_range=(1, 2), sgd__alpha=1e-07 .....................
[CV] ...... tfidf__ngram_range=(1, 2), sgd__alpha=1e-07, total=  10.2s
[CV] tfidf__ngram_range=(1, 2), sgd__alpha=1e-07 .....................
[CV] ...... tfidf__ngram_range=(1, 2), sgd__alpha=1e-07, total=   9.8s
[CV] t

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:  6.6min finished


In [69]:
gs_sgd.best_estimator_.get_params()

{'memory': None,
 'sgd': SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
        eta0=0.0, fit_intercept=True, l1_ratio=0.15,
        learning_rate='optimal', loss='hinge', max_iter=None, n_iter=None,
        n_jobs=1, penalty='l2', power_t=0.5, random_state=42, shuffle=True,
        tol=None, verbose=0, warm_start=False),
 'sgd__alpha': 0.0001,
 'sgd__average': False,
 'sgd__class_weight': None,
 'sgd__epsilon': 0.1,
 'sgd__eta0': 0.0,
 'sgd__fit_intercept': True,
 'sgd__l1_ratio': 0.15,
 'sgd__learning_rate': 'optimal',
 'sgd__loss': 'hinge',
 'sgd__max_iter': None,
 'sgd__n_iter': None,
 'sgd__n_jobs': 1,
 'sgd__penalty': 'l2',
 'sgd__power_t': 0.5,
 'sgd__random_state': 42,
 'sgd__shuffle': True,
 'sgd__tol': None,
 'sgd__verbose': 0,
 'sgd__warm_start': False,
 'steps': [('tfidf',
   TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, m

In [70]:
best_sgd_test_score = gs_sgd.score(test_corpus, test_label_names)
print('Test Accuracy :', best_sgd_test_score)

Test Accuracy : 0.768415825195


In [25]:
import model_evaluation_utils as meu

In [26]:
mnb_predictions = gs_mnb.predict(test_corpus)
unique_classes = list(set(test_label_names))
meu.get_metrics(true_labels=test_label_names, predicted_labels=mnb_predictions)

Accuracy: 0.7735
Precision: 0.7825
Recall: 0.7735
F1 Score: 0.7696


In [27]:
meu.display_classification_report(true_labels=test_label_names, 
                                  predicted_labels=mnb_predictions, classes=unique_classes)

                          precision    recall  f1-score   support

 comp.os.ms-windows.misc       0.76      0.72      0.74       315
      talk.politics.misc       0.72      0.68      0.70       244
           comp.graphics       0.64      0.75      0.69       289
          comp.windows.x       0.79      0.84      0.81       287
      talk.religion.misc       0.67      0.21      0.32       199
comp.sys.ibm.pc.hardware       0.69      0.76      0.72       324
   comp.sys.mac.hardware       0.78      0.77      0.77       295
               sci.crypt       0.79      0.85      0.82       302
   talk.politics.mideast       0.85      0.87      0.86       326
            misc.forsale       0.83      0.77      0.80       314
                 sci.med       0.88      0.88      0.88       322
         rec.motorcycles       0.88      0.74      0.80       351
         sci.electronics       0.80      0.72      0.76       307
        rec.sport.hockey       0.88      0.92      0.90       308
      tal

In [35]:
label_data_map = {v:k for k, v in data_labels_map.items()}
label_map_df = pd.DataFrame(list(label_data_map.items()), columns=['Label Name', 'Label Number'])
label_map_df

,Label Name,Label Number
0,alt.atheism,0
1,comp.graphics,1
2,comp.os.ms-windows.misc,2
3,comp.sys.ibm.pc.hardware,3
4,comp.sys.mac.hardware,4
5,comp.windows.x,5
6,misc.forsale,6
7,rec.autos,7
8,rec.motorcycles,8
9,rec.sport.baseball,9


In [37]:
unique_class_nums = label_map_df['Label Number'].values
mnb_prediction_class_nums = [label_data_map[item] for item in mnb_predictions]
meu.display_confusion_matrix_pretty(true_labels=test_label_nums, 
                                   predicted_labels=mnb_prediction_class_nums, classes=unique_class_nums)

Predicted:                                                         \
                   0    1    2    3    4    5    6    7    8    9    10   11   
Actual: 0         153    1    0    0    0    2    0    3    2    1    3    1   
        1           2  218   10   11    5   16    4    0    2    2    0    4   
        2           1   22  226   24    6   15    3    1    0    1    1    3   
        3           1   15   22  245   20    2    9    3    0    0    0    1   
        4           0    7   10   22  227    5    5    1    0    0    1    9   
        5           1   26   10    4    0  241    1    0    0    0    0    1   
        6           0    2    4   17   13    0  242    7    1    0    1    3   
        7           2    3    3    3    3    2    9  242   16    1    0    4   
        8           0    3    1    0    2    3    6   25  260    2    7    2   
        9           0    3    0    2    2    4    0    0    1  297   12    6   
        10          2    0    0    0    0    3    2    1    1    5  282    1   
        11          0    5    4    0    4    1    0    1    2    1    3  256   
        12          0   14    3   22    5    2    7    5    0    0    1    9   
        13          1    8    0    0    0    1    0    3    0    0    0    0   
        14          2   11    2    1    2    2    2    2    2    2    0    4   
        15          4    2    1    0    2    1    0    0    0    1    0    0   
        16          3    0    0    1    1    1    0    0    4    1    2    9   
        17          2    1    1    1    0    0    0    0    4    1    0    4   
        18          0    1    1    0    0    1    0    1    2    1    5    5   
        19         18    0    1    0    0    4    1    1    0    1    2    4   

                                                   
             12   13   14   15   16   17   18  19  
Actual: 0     0    1    4   57   10   15    5  10  
        1     2    0    9    2    1    0    1   0  
        2     4    3    3    1    1    0    0   0  
        3     5    0    0    0    1    0    0   0  
        4     4    0    0    2    0    0    1   1  
        5     0    0    1    1    1    0    0   0  
        6     8    1    5    4    3    2    1   0  
        7     9    5    3    4   12    3    3   1  
        8     3    6    4    8   10    1    8   0  
        9     0    1    1    3    3    1    0   0  
        10    1    2    1    1    1    2    3   0  
        11    3    0    1    2   13    2    4   0  
        12  222    5    4    3    2    1    2   0  
        13    6  284    5    7    2    2    3   0  
        14    5    5  264    4    8    2    4   0  
        15    0    3    1  295    4    3    2   2  
        16    3    0    2    6  227    6   12   3  
        17    0    1    1    9    5  284   11   1  
        18    0    2    5   14   31    6  166   3  
        19    2    4    1   93   15    4    6  42

In [38]:
unique_classes = label_map_df['Label Name'].values
meu.display_confusion_matrix_pretty(true_labels=test_label_names, 
                                    predicted_labels=mnb_predictions, classes=unique_classes)

Predicted:                \
                                 alt.atheism comp.graphics   
Actual: alt.atheism                      153             1   
        comp.graphics                      2           218   
        comp.os.ms-windows.misc            1            22   
        comp.sys.ibm.pc.hardware           1            15   
        comp.sys.mac.hardware              0             7   
        comp.windows.x                     1            26   
        misc.forsale                       0             2   
        rec.autos                          2             3   
        rec.motorcycles                    0             3   
        rec.sport.baseball                 0             3   
        rec.sport.hockey                   2             0   
        sci.crypt                          0             5   
        sci.electronics                    0            14   
        sci.med                            1             8   
        sci.space                          2            11   
        soc.religion.christian             4             2   
        talk.politics.guns                 3             0   
        talk.politics.mideast              2             1   
        talk.politics.misc                 0             1   
        talk.religion.misc                18             0   

                                                          \
                                 comp.os.ms-windows.misc   
Actual: alt.atheism                                    0   
        comp.graphics                                 10   
        comp.os.ms-windows.misc                      226   
        comp.sys.ibm.pc.hardware                      22   
        comp.sys.mac.hardware                         10   
        comp.windows.x                                10   
        misc.forsale                                   4   
        rec.autos                                      3   
        rec.motorcycles                                1   
        rec.sport.baseball                             0   
        rec.sport.hockey                               0   
        sci.crypt                                      4   
        sci.electronics                                3   
        sci.med                                        0   
        sci.space                                      2   
        soc.religion.christian                         1   
        talk.politics.guns                             0   
        talk.politics.mideast                          1   
        talk.politics.misc                             1   
        talk.religion.misc                             1   

                                                           \
                                 comp.sys.ibm.pc.hardware   
Actual: alt.atheism                                     0   
        comp.graphics                                  11   
        comp.os.ms-windows.misc                        24   
        comp.sys.ibm.pc.hardware                      245   
        comp.sys.mac.hardware                          22   
        comp.windows.x                                  4   
        misc.forsale                                   17   
        rec.autos                                       3   
        rec.motorcycles                                 0   
        rec.sport.baseball                              2   
        rec.sport.hockey                                0   
        sci.crypt                                       0   
        sci.electronics                                22   
        sci.med                                         0   
        sci.space                                       1   
        soc.religion.christian                          0   
        talk.politics.guns                              1   
        talk.politics.mideast                           1   
        talk.politics.misc                              0   
        talk.religion.misc                              0   

     

In [39]:
label_map_df[label_map_df['Label Number'].isin([0, 15, 19])]

,Label Name,Label Number
0,alt.atheism,0
15,soc.religion.christian,15
19,talk.religion.misc,19


In [59]:
train_idx, test_idx = train_test_split(np.array(range(len(data_df['Article']))), test_size=0.33, random_state=42)
test_idx

array([ 4105, 12650,  7039, ...,  4772,  7803,  9616])

In [61]:
predict_probas = gs_mnb.predict_proba(test_corpus).max(axis=1)
test_df = data_df.iloc[test_idx]
test_df['Predicted Name'] = mnb_predictions
test_df['Predicted Confidence'] = predict_probas
test_df.head()

,Article,Clean Article,Target Label,Target Name,Predicted Name,Predicted Confidence
4105,Just a little nitpicking. Wasn't it the government that required\r\na standard railway gauge ? D...,little nitpicking not government require standard railway gauge not improve thing please not mis...,11,sci.crypt,sci.crypt,0.975658
12650,\r\nIt means that the EFF's public stance is complicated with issues irrelevant\r\nto the encryp...,mean eff public stance complicate issue irrelevant encryption issue per se may well people care ...,11,sci.crypt,sci.crypt,0.988600
7039,\r\n\r\n\r\n\r\n\r\nSo after I've flashed my lights at the chap in front and he doesn't\r\n'pass...,flash light chap front not pass next major highway lane direction keep extreme right block folk ...,7,rec.autos,rec.motorcycles,0.729504
3310,: I think most of the problems mainly arose from Manager Gene Mauch's\r\n: ineptitude in managin...,think problem mainly arise manager gene mauchs ineptitude manage pitching staff stretch abuse ji...,9,rec.sport.baseball,rec.sport.baseball,0.999942
16360,"OK... quick scenario... you're at home, not bothering anybody... next thing you\r\nknow, somebod...",ok quick scenario home not bother anybody next thing know somebody come crash upstairs window he...,16,talk.politics.guns,talk.politics.guns,0.998837


In [77]:
pd.set_option('display.max_colwidth', 200)
res_df = (test_df[(test_df['Target Name'] == 'talk.religion.misc') & (test_df['Predicted Name'] == 'soc.religion.christian')]
       .sort_values(by=['Predicted Confidence'], ascending=False).head(5))
res_df

,Article,Clean Article,Target Label,Target Name,Predicted Name,Predicted Confidence
8968,"\r\nZoroaster is far older than Daniel. If anything, one could claim that,\r\nin a sense, Daniel is a descendant of Zoroaster; as Daniel, though being\r\nHebrew, has assimilated into Zoroastrianis...",zoroaster far old daniel anything one could claim sense daniel descendant zoroaster daniel though hebrew assimilate zoroastrianism successfully introduce religion tanakh judaism however majority b...,19,talk.religion.misc,soc.religion.christian,0.999557
3299,"There were some recent developments in the dispute about Masonry among\r\nSouthern Baptists. I posted a summary over in bit.listserv.christia, and\r\nI suppose that it might be useful here. Note...",recent development dispute masonry among southern baptists post summary bit listserv christia suppose may useful note not necessarily agree disagree follow present information short summary southe...,19,talk.religion.misc,soc.religion.christian,0.999384
4367,:\r\n (lots of stuff about the Nicene Creed deleted which can be read in the\r\n original basenote. I will also leave it up to other LDS netters to\r\n take Mr. Weiss to task on using Mormon Do...,lot stuff nicene creed delete read original basenote also leave lds netter take mr weiss task use mormon doctrine declare difinitive word lds church teach doctrine hopefully lds netter amiable exp...,19,talk.religion.misc,soc.religion.christian,0.999254
12608,: >: >> Gilligan = Sloth\r\n: >: >> Skipper = Anger\r\n: >: >> Thurston Howell III = Greed\r\n: >: >> Lovey Howell = Gluttony\r\n: >: >> Ginger = Lust\r\n: >: >> Professor = Pride\r\n: >: >> Mary ...,gilligan sloth skipper anger thurston howell iii greed lovey howell gluttony ginger lust professor pride mary ann envy assorted monkeys secular humanism assorted headhunters godless heathen savage...,19,talk.religion.misc,soc.religion.christian,0.998755
16758,"\r\n\r\nThere were many injustices in the middle ages. And this is truely sad.\r\nI would hate to see a day when churches put people to death or torchured\r\nthem for practicing homosexuality, or...",many injustice middle age truely sad would hate see day church put people death torchur practice homosexuality crime church not call take government world may homosexual treat cruelly today not me...,19,talk.religion.misc,soc.religion.christian,0.998243


In [78]:
pd.set_option('display.max_colwidth', 200)
res_df = (test_df[(test_df['Target Name'] == 'talk.religion.misc') & (test_df['Predicted Name'] == 'alt.atheism')]
       .sort_values(by=['Predicted Confidence'], ascending=False).head(5))
res_df

,Article,Clean Article,Target Label,Target Name,Predicted Name,Predicted Confidence
914,"\r\n\r\nAtoms are not objective. They aren't even real. What scientists call\r\nan atom is nothing more than a mathematical model that describes \r\ncertain physical, observable properties of ou...",atom not objective not even real scientist call atom nothing mathematical model describe certain physical observable property surrounding subjective objective though approach scientist take discus...,19,talk.religion.misc,alt.atheism,0.996075
2334,In <1ren9a$94q@morrow.stanford.edu> salem@pangea.Stanford.EDU (Bruce Salem) \r\n\r\n\r\n\r\nThis brings up another something I have never understood. I asked this once\r\nbefore and got a few int...,renaqmorrow stanford edu salempangea stanford edu bruce salem bring another something never understand ask get interesting response somehow not seem satisfied would nt not consider good source may...,19,talk.religion.misc,alt.atheism,0.996051
11117,"\r\n\r\n\tUnless God admits that he didn't do it....\r\n\r\n\t=)\r\n\r\n\r\n--- \r\n\r\n "" I'd Cheat on Hillary Too.""",unless god admit not would cheat hillary,19,talk.religion.misc,alt.atheism,0.965725
12386,"\r\nAh, you taking everything as literal quotation. No wonder you're confused.\r\n\r\nFirst, can I ask that we decide on a definition of ""objective""?\r\n\r\n\r\nAnd?\r\n\r\n\r\nI'd guess that it ...",ah take everything literal quotation no wonder confused first ask decide definition objective would guess may may case people unable evaluate complex moral issue rather leave behave immorally may ...,19,talk.religion.misc,alt.atheism,0.772658
9360,"\r\nYes, as a philosophy weak atheism is worthless. This is true in\r\nexactly the same sense that as a philosophy Christians' disbelief in\r\nZeus is worthless. Atheists construct their persona...",yes philosophy weak atheism worthless true exactly sense philosophy christian disbelief zeus worthless atheists construct personal philosophy many different source build non god base idea way chri...,19,talk.religion.misc,alt.atheism,0.757788
